In [1]:
import json
from pathlib import Path
import nltk
import csv
from pytorch_pretrained_bert.tokenization import BasicTokenizer
#nltk.download('punkt')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


**Convert BRAT-Format into JSON**

In [2]:
path_ann = Path('test').glob('*.ann')
path_txt = Path('test').glob('*.txt')

d = {}

for txt,ann in zip(path_txt, path_ann): # iterate over files: 
    
    # because path is object not string
    path_t = str(txt) 
    pmid = path_t[path_t.find('/')+1:path_t.find('.txt')]
    #print(pmid)
    d[pmid]={}
    path_a = str(ann)
    file_t = open(path_t, 'r')
    lines_t = file_t.readlines()
    file_t.close()
    file_a = open(path_a, 'r')
    entities = csv.reader(file_a, delimiter='\t')
    
    list_of_entities = []
    for line in entities:
        if line[1].split()[0]=='MiRNA':
            begin = int(line[1].split()[1])
            end = int(line[1].split()[2])
            entity = line[2]
            list_of_entities.append({'entity':entity, 'begin':begin, 'end':end})
    file_a.close()
    abstract = lines_t[0]
    d[pmid]['abstract'] = abstract
    #split abstract into sentences:
    list_of_sentences = nltk.tokenize.sent_tokenize(abstract)
    d[pmid]['sentences'] = []
    begin_sent = 0
    for sentence in list_of_sentences:
        end_sent = begin_sent + len(sentence)
        
        # iterate over entities and check wheter the offsets match
        entities_per_sentence = []
        for ent_d in list_of_entities:
            if ent_d['begin']>= begin_sent and ent_d['end']<= end_sent:
                entities_per_sentence.append(ent_d)
        
        if len(entities_per_sentence)!=0:
            d[pmid]['sentences'].append({'sentence':sentence, 'begin':begin_sent, 'end':end_sent, 
                                        'entities':entities_per_sentence})
        else:
            d[pmid]['sentences'].append({'sentence':sentence, 'begin':begin_sent, 'end':end_sent})
        
        
        begin_sent = end_sent + 1

In [3]:
f = open('miRTex_test.json', 'w', encoding='utf-8')
print(json.dumps(d, indent=3, ensure_ascii=False), file=f)
f.close()

**Convert BRAT Format into BERT Format**

In [89]:
path_ann = Path('development').glob('*.ann')
path_txt = Path('development').glob('*.txt')

sentences = []
labels = []
c = 0
for txt,ann in zip(path_txt, path_ann): # iterate over files: 
    
    # because path is object not string
    path_t = str(txt) 
    pmid = path_t[path_t.find('/')+1:path_t.find('.txt')]
    path_a = str(ann)
    file_t = open(path_t, 'r')
    lines_t = file_t.readlines()
    file_t.close()
    file_a = open(path_a, 'r')
    entities = csv.reader(file_a, delimiter='\t')
    
    list_of_entities = []
    for line in entities:
        if line[1].split()[0]=='MiRNA':
            begin = int(line[1].split()[1])
            end = int(line[1].split()[2])
            entity = line[2]
            list_of_entities.append({'entity':entity, 'begin':begin, 'end':end})
    file_a.close()

    abstract = lines_t[0]
    #split abstract into sentences:
    list_of_sentences = nltk.tokenize.sent_tokenize(abstract)
    #print(list_of_sentences)
    begin_sent = 0
    
    for sentence in list_of_sentences:
        splitted_sentence = sentence.replace(',',' ,').replace('-regulated',' -regulated').replace('-LV', ' -LV').replace('Pre-', 'Pre- ').replace('Anti-', 'Anti- ').replace('-mediated', ' -mediated').replace('-HDAC4', ' -HDAC4').replace('mTOR-', 'mTOR- ').replace('Rawq01+', 'Rawq01+ ').replace('-ICAM-1', ' -ICAM-1').replace(';', ' ;').replace('anti-', 'anti- ').replace('/', ' / ').replace('as-', 'as- ').replace('-induced', ' -induced').replace('pre-', ' pre- ').replace('.', ' .').replace('(', ' ( ').replace(')', ' )').replace('-reducing', ' -reducing').replace(':', ' :').replace('-transfected', ' -transfected').split()
        temp_labels = ['O' for i in range(len(splitted_sentence))]
        #print(splitted_sentence)
        #print(temp_labels)
        end_sent = begin_sent + len(sentence)
        
        # iterate over entities and check wheter the offsets match
        entities_per_sentence = []
        for ent_d in list_of_entities:
            if ent_d['begin']>= begin_sent and ent_d['end']<= end_sent:
                entities_per_sentence.append(ent_d)
        
        if len(entities_per_sentence)!=0:
            index = 0
            for ent in entities_per_sentence:
                #print(ent['entity'], index)
                #print(splitted_sentence, splitted_sentence[index:])
                
              #  if ent['entity'] not in splitted_sentence:
                if ' ' in ent['entity'] or '/' in ent['entity']:
                    e_splitted = ent['entity'].split()
                    if e_splitted[0] in splitted_sentence:
                        index = splitted_sentence.index(e_splitted[0])
                        temp_labels[index]='B'
                        for ind in range(1, len(e_splitted)):
                            temp_labels[index]=='I'
                    
                elif ent['entity'] in splitted_sentence[index:]:
                    index = splitted_sentence[index:].index(ent['entity'])+index
                    temp_labels[index]='B'
                elif ent['entity'] in splitted_sentence:
                    index = splitted_sentence.index(ent['entity'])
                    temp_labels[index]='B'
                
                else:                      
                    print(ent['entity'])
                    print(splitted_sentence)
                    print(pmid)
                    c += 1
                    continue
                
                
                #print(index)
            #print(entities_per_sentence)
        
        begin_sent = end_sent + 1
        sentences.append(splitted_sentence)
        labels.append(temp_labels)
        #print(splitted_sentence)
        #print(temp_labels)
    #break
print(c)

### for the remaining ones, I did it manually ... 

(miR)-17-92
['For', 'example', ',', 'the', 'miRNA', '(', 'miR', ')-17-92', 'cluster', 'has', 'been', 'characterized', 'as', 'an', 'oncogene', ',', 'while', 'let-7', 'represses', 'Ras', 'and', 'miR-15a', '/', '-16-1', 'represses', 'Bcl-2', ',', 'thereby', 'acting', 'as', 'tumor', 'suppressors', '.']
16784027
(miR)-124a
['Here', ',', 'we', 'demonstrate', 'that', 'human', 'IkappaBzeta', 'is', 'posttranscriptionally', 'regulated', 'by', 'microRNA', '(', 'miR', ')-124a', '.']
19502795
micro(mi)RNA-146a
['We', 'also', 'investigated', 'the', 'changes', 'in', 'micro', '(', 'mi', ')RNA-146a', ',', 'miRNA-155', ',', 'and', 'TNF', 'receptor', '1', '(', 'TNFR1', ')', 'expression', 'after', 'stimulation', '.']
22876745
hsa-miR-196a2*
['Manipulating', 'the', 'hsa-miR-196a2*-TP63', 'axis', 'might', 'provide', 'a', 'potential', 'tumor-suppressive', 'strategy', 'to', 'alleviate', 'the', 'aggressive', 'behavior', 'and', 'poor', 'prognosis', 'of', 'some', 'ERalpha-positive', 'as', 'well', 'as', 'many', '

In [97]:
len(labels[0])
print(labels[5])
print(sentences[5])

['O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['The', 'most', 'interesting', 'results', 'came', 'from', 'miR-221', ',', 'strongly', 'up', '-regulated', 'in', 'glioblastoma', 'and', 'from', 'a', 'set', 'of', 'brain-enriched', 'miRNAs', ',', 'miR-128', ',', 'miR-181a', ',', 'miR-181b', ',', 'and', 'miR-181c', ',', 'which', 'are', 'down', '-regulated', 'in', 'glioblastoma', '.']


In [99]:
def tokenize_sentences_and_labels(labels, sent):

    tokenizer = BasicTokenizer(do_lower_case=False)

    tokenized_labels = []
    tokenized_sentences = []

    for i in range(len(sent)):
        tokenized_labels_temp = []
        tokenized_sentences_temp = []
        #for word, label in zip(sent[i].split(), labels[i]):
        for word, label in zip(sent[i], labels[i]):
            if len(tokenizer.tokenize(word)) > 1:
                temp_words = tokenizer.tokenize(word)
                tokenized_sentences_temp.append(temp_words[0])
                tokenized_labels_temp.append(label)
                for w in range(1, len(temp_words)):
                    tokenized_sentences_temp.append(temp_words[w])
                    if label == 'B':
                        tokenized_labels_temp.append('I')
                    else:
                        tokenized_labels_temp.append('O')

            else:
                tokenized_sentences_temp.append(word)
                tokenized_labels_temp.append('O')

        tokenized_sentences.append(tokenized_sentences_temp)
        tokenized_labels.append(tokenized_labels_temp)

    return tokenized_sentences, tokenized_labels

In [100]:
s, l = tokenize_sentences_and_labels(labels, sentences)

In [102]:
def save_tsv_file(output, sentences, labels):
    with open(output, 'w', encoding='utf-8') as f:
        for i in range(len(labels)):
            for word, label in zip(sentences[i], labels[i]):
                f.write("%s\t%s\n" % (word, label))
            f.write('\n')

In [103]:
save_tsv_file('test_miRTex.tsv', s, l)